In [2]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
def get_spectr(np_wav_files, nperseg=512, samplerate=24000):
    output = list()
    for i, wav in enumerate(np_wav_files):
        _, _, spect = signal.stft(wav, samplerate, nperseg=nperseg)
        output.append(spect)
        if i % 500 == 0: print(i)
    return output

In [ ]:
spect_wav = get_spectr(np_wav_files)

In [ ]:
def get_pkl(file_path):
    with open(file_path, 'rb') as file:
        loaded_list = pickle.load(file)
    return loaded_list

In [1]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def process_data(np_wav, nperseg=512, samplerate=24000):
    # Function to process a single piece of data
    # Implement your custom processing logic here
    _, _, processed_data = signal.stft(np_wav, samplerate, nperseg=nperseg)  # Example processing: doubling the data
    return processed_data

def collate_fn(batch):
    processed_batch = []
    for data in batch:
        processed_data = process_data(data)
        processed_batch.append(processed_data)
    return torch.stack(processed_batch)

data = [1, 2, 3, 4, 5]  # Example data

dataset = CustomDataset(data)
dataloader = DataLoader(dataset, batch_size=2, collate_fn=collate_fn) # + num thread num_workers=6,

for batch in dataloader:
    print(batch)

1
